In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import timm

import torch
from tqdm.notebook import tqdm
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
from torchvision.utils import make_grid
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch.optim as optim

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")

In [4]:
root_dir = "Y:\Python\Datasets\Whales\\train_images"
path = os.listdir(root_dir)

In [5]:
df = pd.read_csv(r"Y:\Python\Datasets\Whales\train.csv")
target = df.species
data = df.image
id_ = df.individual_id
class_list = [i for i in df.species]
num_classes = len(set(class_list))

In [6]:
df1 = df.individual_id.unique

In [7]:
species_name = [i for i in set(class_list)]
species_index = [i for i in range(30)]
whale_dict = {species_name[i]: species_index[i] for i in range(len(species_name))}

all_y = []
for i in target:
    for key, value in whale_dict.items():
        if i == key:
            all_y.append(value)

In [8]:
all_class_list = []
for i in df.species:
    for key, value in whale_dict.items():
        if key == i:
            all_class_list.append(value)

In [9]:
df["class_id"] = all_class_list

In [10]:
all_y = np.array(all_y)
all_X = []
img_list = [x for x in path]
for filename in path:
    all_X.append(os.path.join(root_dir, filename))

In [11]:
train_df, test_df = train_test_split(df, stratify = df.class_id, random_state = 666)

In [12]:
train_df

,image,species,individual_id,class_id
38738,c2a200933da5cc.jpg,bottlenose_dolphin,599bf01ce321,13
2939,0ee1fac5cdf8b4.jpg,blue_whale,e47303f9fb11,27
12856,40cc2b309e8f9c.jpg,fin_whale,f3d9d1f9eece,29
28755,90d92013420d8a.jpg,beluga,dba4e482f0ad,23
19607,62ea8e33250e59.jpg,fin_whale,ceb953072484,29
...,...,...,...,...
5506,1b8badc12da97c.jpg,bottlenose_dolpin,7b0ef9435d31,18
3355,10fec58dce99f5.jpg,bottlenose_dolphin,30ca1c6cb592,13
38417,c0f9a535b4be6a.jpg,fin_whale,87a3edd9e541,29
9902,31d34a240eead8.jpg,minke_whale,53741c7362b8,17


In [11]:
#np.unique(train_y).shape

In [12]:
def data_augment():
    augmented_data = A.Compose([
                            A.Resize(256,256),
                            A.Normalize(),
                            ToTensorV2()
    ])
    
    return augmented_data

In [13]:
class WhaleSet(Dataset):
    def __init__(self, root_dir, df, transform = None):
        super().__init__()
        self.root_dir = root_dir
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
        
    def __getitem__(self, idx):
        target = self.df.iloc[idx].class_id
        image = cv2.imread(os.path.join(self.root_dir, self.df.iloc[idx].image))
        if self.transform is not None:
            image = self.transform(image=image)["image"]

        return image, target

In [1]:
df

NameError: name 'df' is not defined

In [24]:
train_dataset = WhaleSet(root_dir, train_df, data_augment())
train_loader = DataLoader(train_dataset, batch_size = 2, shuffle = True)

test_dataset = WhaleSet(root_dir, test_df, data_augment())
test_loader = DataLoader(train_dataset, batch_size = 1)

In [16]:
model = timm.create_model('tf_efficientnet_b5', num_classes = num_classes, in_chans = 3, pretrained = True)

In [17]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduce = reduce

    def forward(self, inputs, targets):
        BCE_loss = nn.CrossEntropyLoss()(inputs, targets)

        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [18]:
loss_fn = FocalLoss().to(device)
optimizer = torch.optim.Adam(model.parameters())
num_epochs = 300

In [19]:
from torch.utils.tensorboard import SummaryWriter

In [20]:
#writer = SummaryWriter('Y:\\Python\\Experiments\\dolf_1')

In [21]:
train_loss = []
val_loss = []
j = 0
k = 0
i = 0
best_loss = 10000
model.train()
model.cuda()
for epoch in range(num_epochs):
    for X, y in tqdm(train_loader):
        optimizer.zero_grad()
        pred = model(torch.tensor(X).to(device))
        loss = loss_fn(pred, torch.tensor(y).to(device).long())
        loss.backward()
        optimizer.step()
        #writer.add_scalar('training loss', loss.item(), global_step=j)
        #j+=1
        train_loss.append(loss.item())
    
    for X, y in tqdm(test_loader):
        pred = model(torch.tensor(X).to(device))
        loss = loss_fn(pred, torch.tensor(y).to(device).long())
        
        val_loss.append(loss.item())
        #writer.add_scalar('validation loss', loss.item(), global_step=k)
        #k+=1
    i += 1   
    #if best_loss > np.mean(val_loss):
    #    best_loss = np.mean(val_loss)
    #    torch.save(model.state_dict(), "model.pt")
    print("Epoch: ", i)
    print("train loss: ", np.mean(train_loss))
    print("val loss: ", np.mean(val_loss))

  0%|          | 0/19137 [00:00<?, ?it/s]

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


  0%|          | 0/19137 [00:00<?, ?it/s]

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch:  1
train loss:  0.8766444494198878
val loss:  0.4850831779856972


...

  0%|          | 0/19137 [00:00<?, ?it/s]

  0%|          | 0/19137 [00:00<?, ?it/s]

Epoch:  14
train loss:  0.20144096686639998
val loss:  0.14613473296180735


  0%|          | 0/19137 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [22]:
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

conv_stem.weight 	 torch.Size([48, 3, 3, 3])
bn1.weight 	 torch.Size([48])
bn1.bias 	 torch.Size([48])
bn1.running_mean 	 torch.Size([48])
bn1.running_var 	 torch.Size([48])
bn1.num_batches_tracked 	 torch.Size([])
blocks.0.0.conv_dw.weight 	 torch.Size([48, 1, 3, 3])
blocks.0.0.bn1.weight 	 torch.Size([48])
blocks.0.0.bn1.bias 	 torch.Size([48])
blocks.0.0.bn1.running_mean 	 torch.Size([48])
blocks.0.0.bn1.running_var 	 torch.Size([48])
blocks.0.0.bn1.num_batches_tracked 	 torch.Size([])
blocks.0.0.se.conv_reduce.weight 	 torch.Size([12, 48, 1, 1])
blocks.0.0.se.conv_reduce.bias 	 torch.Size([12])
blocks.0.0.se.conv_expand.weight 	 torch.Size([48, 12, 1, 1])
blocks.0.0.se.conv_expand.bias 	 torch.Size([48])
blocks.0.0.conv_pw.weight 	 torch.Size([24, 48, 1, 1])
blocks.0.0.bn2.weight 	 torch.Size([24])
blocks.0.0.bn2.bias 	 torch.Size([24])
blocks.0.0.bn2.running_mean 	 torch.Size([24])
blocks.0.0.bn2.running_var 	 torch.Size([24])
blocks.0.0.bn2.num_batches_tracked 	 torch.Size([])
bl

blocks.2.1.conv_pwl.weight 	 torch.Size([64, 384, 1, 1])
blocks.2.1.bn3.weight 	 torch.Size([64])
blocks.2.1.bn3.bias 	 torch.Size([64])
blocks.2.1.bn3.running_mean 	 torch.Size([64])
blocks.2.1.bn3.running_var 	 torch.Size([64])
blocks.2.1.bn3.num_batches_tracked 	 torch.Size([])
blocks.2.2.conv_pw.weight 	 torch.Size([384, 64, 1, 1])
blocks.2.2.bn1.weight 	 torch.Size([384])
blocks.2.2.bn1.bias 	 torch.Size([384])
blocks.2.2.bn1.running_mean 	 torch.Size([384])
blocks.2.2.bn1.running_var 	 torch.Size([384])
blocks.2.2.bn1.num_batches_tracked 	 torch.Size([])
blocks.2.2.conv_dw.weight 	 torch.Size([384, 1, 5, 5])
blocks.2.2.bn2.weight 	 torch.Size([384])
blocks.2.2.bn2.bias 	 torch.Size([384])
blocks.2.2.bn2.running_mean 	 torch.Size([384])
blocks.2.2.bn2.running_var 	 torch.Size([384])
blocks.2.2.bn2.num_batches_tracked 	 torch.Size([])
blocks.2.2.se.conv_reduce.weight 	 torch.Size([16, 384, 1, 1])
blocks.2.2.se.conv_reduce.bias 	 torch.Size([16])
blocks.2.2.se.conv_expand.weight 	 t

blocks.3.4.bn3.bias 	 torch.Size([128])
blocks.3.4.bn3.running_mean 	 torch.Size([128])
blocks.3.4.bn3.running_var 	 torch.Size([128])
blocks.3.4.bn3.num_batches_tracked 	 torch.Size([])
blocks.3.5.conv_pw.weight 	 torch.Size([768, 128, 1, 1])
blocks.3.5.bn1.weight 	 torch.Size([768])
blocks.3.5.bn1.bias 	 torch.Size([768])
blocks.3.5.bn1.running_mean 	 torch.Size([768])
blocks.3.5.bn1.running_var 	 torch.Size([768])
blocks.3.5.bn1.num_batches_tracked 	 torch.Size([])
blocks.3.5.conv_dw.weight 	 torch.Size([768, 1, 3, 3])
blocks.3.5.bn2.weight 	 torch.Size([768])
blocks.3.5.bn2.bias 	 torch.Size([768])
blocks.3.5.bn2.running_mean 	 torch.Size([768])
blocks.3.5.bn2.running_var 	 torch.Size([768])
blocks.3.5.bn2.num_batches_tracked 	 torch.Size([])
blocks.3.5.se.conv_reduce.weight 	 torch.Size([32, 768, 1, 1])
blocks.3.5.se.conv_reduce.bias 	 torch.Size([32])
blocks.3.5.se.conv_expand.weight 	 torch.Size([768, 32, 1, 1])
blocks.3.5.se.conv_expand.bias 	 torch.Size([768])
blocks.3.5.conv_

blocks.4.6.bn3.num_batches_tracked 	 torch.Size([])
blocks.5.0.conv_pw.weight 	 torch.Size([1056, 176, 1, 1])
blocks.5.0.bn1.weight 	 torch.Size([1056])
blocks.5.0.bn1.bias 	 torch.Size([1056])
blocks.5.0.bn1.running_mean 	 torch.Size([1056])
blocks.5.0.bn1.running_var 	 torch.Size([1056])
blocks.5.0.bn1.num_batches_tracked 	 torch.Size([])
blocks.5.0.conv_dw.weight 	 torch.Size([1056, 1, 5, 5])
blocks.5.0.bn2.weight 	 torch.Size([1056])
blocks.5.0.bn2.bias 	 torch.Size([1056])
blocks.5.0.bn2.running_mean 	 torch.Size([1056])
blocks.5.0.bn2.running_var 	 torch.Size([1056])
blocks.5.0.bn2.num_batches_tracked 	 torch.Size([])
blocks.5.0.se.conv_reduce.weight 	 torch.Size([44, 1056, 1, 1])
blocks.5.0.se.conv_reduce.bias 	 torch.Size([44])
blocks.5.0.se.conv_expand.weight 	 torch.Size([1056, 44, 1, 1])
blocks.5.0.se.conv_expand.bias 	 torch.Size([1056])
blocks.5.0.conv_pwl.weight 	 torch.Size([304, 1056, 1, 1])
blocks.5.0.bn3.weight 	 torch.Size([304])
blocks.5.0.bn3.bias 	 torch.Size([304

blocks.6.0.bn1.weight 	 torch.Size([1824])
blocks.6.0.bn1.bias 	 torch.Size([1824])
blocks.6.0.bn1.running_mean 	 torch.Size([1824])
blocks.6.0.bn1.running_var 	 torch.Size([1824])
blocks.6.0.bn1.num_batches_tracked 	 torch.Size([])
blocks.6.0.conv_dw.weight 	 torch.Size([1824, 1, 3, 3])
blocks.6.0.bn2.weight 	 torch.Size([1824])
blocks.6.0.bn2.bias 	 torch.Size([1824])
blocks.6.0.bn2.running_mean 	 torch.Size([1824])
blocks.6.0.bn2.running_var 	 torch.Size([1824])
blocks.6.0.bn2.num_batches_tracked 	 torch.Size([])
blocks.6.0.se.conv_reduce.weight 	 torch.Size([76, 1824, 1, 1])
blocks.6.0.se.conv_reduce.bias 	 torch.Size([76])
blocks.6.0.se.conv_expand.weight 	 torch.Size([1824, 76, 1, 1])
blocks.6.0.se.conv_expand.bias 	 torch.Size([1824])
blocks.6.0.conv_pwl.weight 	 torch.Size([512, 1824, 1, 1])
blocks.6.0.bn3.weight 	 torch.Size([512])
blocks.6.0.bn3.bias 	 torch.Size([512])
blocks.6.0.bn3.running_mean 	 torch.Size([512])
blocks.6.0.bn3.running_var 	 torch.Size([512])
blocks.6.0.b

In [215]:
pred_s = []
target_s = []
top_5_preds = []
model.cuda()
model.eval()
for X,y in tqdm(test_loader):
    pred = model(torch.tensor(X).to(device))
    pred_s.append((torch.argmax(pred, dim = 1).detach().cpu().numpy()))
    target_s.append((y.detach().cpu().numpy()))
    top_5_preds.append(torch.topk((torch.softmax(pred, dim = 1).detach().cpu()),
                                  5, dim=1, largest = True, sorted = True))

  0%|          | 0/38274 [00:00<?, ?it/s]

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


KeyboardInterrupt: 

In [209]:
top_5_list = []
for i in p_reds[j]:
    top_5_list.append(torch.topk(p_reds[j]), 5, dim = 1, largest=True,sorted=True)

TypeError: topk(): argument 'input' (position 1) must be Tensor, not list

In [222]:
type(t_arget[1])

torch.Tensor

In [221]:
model.cuda()
model.eval()
p_reds = []
t_arget = []
for X,y in tqdm(test_loader):
        pred = model(torch.tensor(X).to(device))
        t_arget.append(y.detach().cpu().numpy())
        top5.append(torch.softmax(pred, dim = 1).detach().cpu().numpy())
        p_reds.append(sorted(torch.softmax(pred, dim=1).detach().cpu().numpy().reshape(30,1)))

  0%|          | 0/38274 [00:00<?, ?it/s]

c:\users\admin\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import average_precision_score

In [ ]:
print(np.concatenate(pred_s))
print(np.concatenate(target_s))


In [103]:
def mAP5(top5, y):
    N = 0
    Z = 0
    hit_list = []
    for i in range(len(y)):
        if int(y[i]) in top5[i].indices:
            hit_list.append((Z+1)/(N+1))
            Z+=1
        else:
            hit_list.append(0/(N+1))
        N+=1
    return sum(hit_list)/N

In [104]:
def mAP(X, y):
    N = 0
    Z = 0
    hit_list = []
    for i in range(len(y)):
        if X[i] == y[i]:
            hit_list.append((Z+1)/(N+1))
            Z+=1
        else:
            hit_list.append(0/(N+1))
        N+=1
    return sum(hit_list)/N

In [100]:
N = 0
Z = 0
hit_list = []
for i in range(50):
    if int(target_s[i]) in top_5_preds[i].indices:
        hit_list.append((Z+1)/(N+1))
        Z+=1
    else:
        hit_list.append(0/(N+1))
    N+=1

In [139]:
type(top_5_preds[1].item())

AttributeError: 'torch.return_types.topk' object has no attribute 'item'

In [132]:
a = torch.randn(5, 2)
for i in a:
    print(i)

tensor([0.9621, 2.4651])
tensor([0.9541, 0.6222])
tensor([0.2820, 0.7552])
tensor([ 0.9694, -0.1045])
tensor([-0.4733, -1.3888])


In [ ]:
all_pred = []
all_target = []
for X, y in tqdm(test_loader):
    model.cuda()
    with torch.no_grad():
        pred = model(torch.tensor(X).to(device))
        all_pred.append(torch.argmax(pred, dim = 1).detach().cpu().numpy())
        all_target.append(y.detach().cpu().numpy())

In [ ]:
all_pred = np.concatenate(pred_s)
all_target = np.concatenate(target_s)

In [ ]:
report = classification_report(all_target, all_pred)
print(report)